## Water Quality Prediction Project Proposal
### Introduction

Water is one of the main basic needs humans need to survive, though many regions around the globe have limited access to clean, safe drinking-water. A body of water's potability can be determined using a range of water quality metrics, each serving as indicators as to whether or not the water is safe to drink, or in other words, potable.

For this project, we will be utilizing the 'water_potability.csv' file retrieved from the [Water Quality dataset](https://www.kaggle.com/datasets/adityakadiwal/water-potability/data) on Kaggle, which consists of water quality metrics from 3276 distinct water bodies. 

<br>


Our **objective** with this project is to answer this question:


> **Can we predict the potability of water based on its quality metrics?**
   

<br>

The **variables** of this dataset are detailed below:

- **pH Value:** A measure of how acidic or alkaline water is (values fall within the WHO's recommended limits).

- **Hardness:** Indicates the presence of calcium and magnesium salts, which are vital determinants of water's ability to precipitate soap.

- **Solids (Total Dissolved Solids - TDS):** Denotes the concentration of dissolved minerals in water, affecting its taste and appearance.

- **Chloramines:** The concentration of disinfectants, primarily used in public water systems (chlorine and chloramine). Chlorine levels up to 4mg/L are considered safe in drinking water.

- **Sulfate:** Found in many natural sources (groundwater, plants, food, etc.). Concentration varies greatly.

- **Conductivity:** A measure of water's ability to conduct electric current, primarily determined by the amount of dissolved solids in water. Pure water is not a good conductor, and WHO standards state that EC (electrical conductivity) should not exceed 400 μS/cm. 

- **Organic_carbon (Total Organic Carbon - TOC):** A measure of the total amount of carbon in organic compounds in pure water. Comes from decaying natural organic matter and synthetic sources. 

- **Trihalomethanes (THMs):** Chemicals that might be present in water that has been treated with chlorine. Concentration fluctuates according to level of organic material in the water, the temperature of the water, and the amount of chlorine needed to treat the water.

- **Turbidity:** The amount of solid matter suspended in water, influencing the water's transparency. A measure of light emitting properties of water. WHO recommends a value of 5.00 NTU.

- **Potability:** Indicates if water is safe for human consumption or not. '1' is potable, '0' is not potable. 

(389 words)


### Preliminary exploratory data analysis
To begin, we read the data from the web into Python and imported everything we could potentially require. Due to Kaggle's security/authentication methods, we uploaded the dataset file to Google Drive and created a share link with public access. This way, we did not have to directly upload the file into Jupyter or utilize the Kaggle API command. We then went on to clean and wrangle the data and split the data into training and testing sets.

In [1]:
#import commands
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.impute import SimpleImputer

# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

In [2]:
raw_water_data = pd.read_csv('https://drive.google.com/uc?id=13N4nBi8cZCQUQambCexi0-XArwSghdrj')

missing_sum = raw_water_data.isnull().sum()

print(missing_sum)

total_rows = raw_water_data.shape

print(total_rows)

percent_missing = ((missing_sum["ph"] + missing_sum["Sulfate"] + missing_sum["Trihalomethanes"]) / total_rows) * 100

print(percent_missing)

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64
(3276, 10)
[   43.77289377 14340.        ]


In [3]:
preprocessor_missing = make_column_transformer(
    (SimpleImputer(), ["ph", "Sulfate", "Trihalomethanes"]),
    remainder='passthrough',
    verbose_feature_names_out=False
)

preprocessor_missing.fit(raw_water_data)
water_data = preprocessor_missing.transform(raw_water_data)


water_data["Potability"] = water_data["Potability"].replace({
    0: "Not Potable",
    1: "Potable"
})

water_data.isnull().sum()

ph                 0
Sulfate            0
Trihalomethanes    0
Hardness           0
Solids             0
Chloramines        0
Conductivity       0
Organic_carbon     0
Turbidity          0
Potability         0
dtype: int64

In [4]:
water_data["Potability"].value_counts()

Not Potable    1998
Potable        1278
Name: Potability, dtype: int64

In [5]:
np_water = water_data[water_data["Potability"] == "Not Potable"]
p_water = water_data[water_data["Potability"] == "Potable"]
p_water_upsampled = resample(
    p_water, n_samples=np_water.shape[0]
)
upsampled_water = pd.concat((p_water_upsampled, np_water))
upsampled_water['Potability'].value_counts()

Potable        1998
Not Potable    1998
Name: Potability, dtype: int64

In [6]:
water_data = upsampled_water.reset_index(drop=True)
water_data.head(10)

,ph,Sulfate,Trihalomethanes,Hardness,Solids,Chloramines,Conductivity,Organic_carbon,Turbidity,Potability
0,6.000723,301.307173,71.002217,161.002305,21111.664104,4.339126,384.753298,9.261965,4.176219,Potable
1,7.174135,287.085679,74.405616,203.408935,20401.102461,7.681806,315.549900,14.533510,3.939896,Potable
2,7.371914,324.546262,62.677756,148.193698,42059.380417,7.966710,544.848432,17.166504,4.338957,Potable
3,4.429379,333.775777,66.396293,256.212565,8746.901717,9.010332,432.107935,16.307474,4.271168,Potable
4,5.023381,409.918702,71.983851,136.976879,15040.841478,7.588021,454.884809,14.245091,4.477351,Potable
5,6.603641,316.253503,68.872497,206.919743,14598.025787,6.452141,582.110606,12.520933,3.403798,Potable
6,7.080795,303.961739,51.374239,221.673134,32269.504938,8.765238,357.358567,12.819741,4.804829,Potable
7,7.710960,390.596392,58.241734,181.938126,6552.626314,5.389249,516.284083,17.010785,4.288947,Potable
8,6.569054,327.697541,65.750173,208.731454,29123.708304,6.765118,308.086050,12.360292,4.204699,Potable
9,5.763773,333.775777,63.299757,183.073629,22025.696606,8.952896,376.581389,10.547398,3.253200,Potable


In [7]:
water_train, water_test = train_test_split(
    water_data, train_size=0.75, stratify=water_data["Potability"]
)
pd.to_numeric(water_train["ph"])
pd.to_numeric(water_train["ph"])
print(water_train.info())
print(water_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2997 entries, 684 to 1381
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2997 non-null   float64
 1   Sulfate          2997 non-null   float64
 2   Trihalomethanes  2997 non-null   float64
 3   Hardness         2997 non-null   float64
 4   Solids           2997 non-null   float64
 5   Chloramines      2997 non-null   float64
 6   Conductivity     2997 non-null   float64
 7   Organic_carbon   2997 non-null   float64
 8   Turbidity        2997 non-null   float64
 9   Potability       2997 non-null   object 
dtypes: float64(9), object(1)
memory usage: 257.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 3084 to 2023
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               999 non-null    float64
 1   Sulfate          999 non-nul

In [13]:
predictor_vals_summary = water_train.describe()
predictor_vals_summary

,ph,Sulfate,Trihalomethanes,Hardness,Solids,Chloramines,Conductivity,Organic_carbon,Turbidity
count,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000
mean,7.080383,332.517145,66.656199,196.189073,22095.114268,7.096819,425.121606,14.284189,3.972609
std,1.431125,36.880614,15.801546,33.244290,8834.076489,1.601590,80.390805,3.321916,0.777258
min,0.000000,129.000000,0.738000,47.432000,320.942611,0.352000,181.483754,2.200000,1.450000
25%,6.304769,314.970495,56.839602,176.753500,15758.740621,6.084133,365.439202,12.127254,3.445756
50%,7.080795,333.775777,66.396293,196.891061,21046.448517,7.094066,422.017356,14.189856,3.961731
75%,7.822492,349.949530,77.227003,215.940695,27312.168385,8.117218,480.680392,16.506658,4.497529
max,14.000000,481.030642,124.000000,317.338124,61227.196008,13.127000,672.556999,28.300000,6.739000


In [15]:
selected_predictors_summary = water_train[["Solids", "Conductivity", "Hardness", "Organic_carbon", "Chloramines"]].describe()
selected_predictors_summary

,Solids,Conductivity,Hardness,Organic_carbon,Chloramines
count,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000
mean,22095.114268,425.121606,196.189073,14.284189,7.096819
std,8834.076489,80.390805,33.244290,3.321916,1.601590
min,320.942611,181.483754,47.432000,2.200000,0.352000
25%,15758.740621,365.439202,176.753500,12.127254,6.084133
50%,21046.448517,422.017356,196.891061,14.189856,7.094066
75%,27312.168385,480.680392,215.940695,16.506658,8.117218
max,61227.196008,672.556999,317.338124,28.300000,13.127000


### Method
Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results

Although all of the variables do contribute in classifying the potability of water, we decided to pick the five most contributing variable by looking at their relative difference. We have first computed the mean for both potable and not potable variables (first and second pandas series). Then, we calculated the relative difference to see the five most contributing variable (third pandas series). Judging from the values in the third pandas series, we decided to pick "Solids", "Conductivity", "Hardness", "Organic_carbon" and "Chloramines" for the predictor variables and "Potability" for the response variable. 

We will visualize the result by creating a confusion matrix and the plot of estimated accuracy versus the number of neighbors to evaluate our model. We will also find the accuracy for the testing dataset as well as the precision and recall of the model.


(136 words)


In [40]:
np_water = water_data[water_data["Potability"] == "Not Potable"].mean(numeric_only=True)
display(np_water)

p_water = water_data[water_data["Potability"] == "Potable"].mean(numeric_only=True)
display(p_water)

((np_water - p_water)/np_water).abs().nlargest(5)

ph                     7.084658
Sulfate              334.371700
Trihalomethanes       66.308522
Hardness             196.733292
Solids             21777.490788
Chloramines            7.092175
Conductivity         426.730454
Organic_carbon        14.364335
Turbidity              3.965800
dtype: float64

ph                     7.088348
Sulfate              333.959103
Trihalomethanes       66.303303
Hardness             194.252812
Solids             22333.871639
Chloramines            7.140905
Conductivity         420.489666
Organic_carbon        14.252881
Turbidity              3.970928
dtype: float64

Solids            0.025548
Conductivity      0.014625
Hardness          0.012608
Organic_carbon    0.007759
Chloramines       0.006871
dtype: float64